### Chat History with Memory 🧠

<img src="./Image/ChatHistory.png" width="700" height="400" style="display: block; margin: auto;">

### Load ENV file

In [1]:
from dotenv import load_dotenv
import os
load = load_dotenv('./../.env')

print(os.getenv("LANGSMITH_API_KEY"))

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


#### Create LLM object

In [2]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    base_url="http://localhost:11434",
    model = "deepseek-r1:8b",
    temperature=0.5,
    max_tokens = 250
)

In [1]:
%pip install langchain_community

  Using cached langchain_community-0.3.27-py3-none-any.whl.metadata (2.9 kB)
  Using cached aiohttp-3.12.15-cp313-cp313-win_amd64.whl.metadata (7.9 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached pydantic_settings-2.10.1-py3-none-any.whl.metadata (3.4 kB)
  Using cached httpx_sse-0.4.1-py3-none-any.whl.metadata (9.4 kB)
  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiosignal-1.4.0-py3-none-any.whl.metadata (3.7 kB)
  Using cached frozenlist-1.7.0-cp313-cp313-win_amd64.whl.metadata (19 kB)
  Using cached multidict-6.6.3-cp313-cp313-win_amd64.whl.metadata (5.4 kB)
  Using cached propcache-0.3.2-cp313-cp313-win_amd64.whl.metadata (12 kB)
  Using cached yarl-1.20.1-cp313-cp313-win_amd64.whl.metadata (76 kB)
  Using cached marshmallow-3.26.1-py3-none-any.whl.metadata (7.3 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached mypy_extensions-1.1.0-py3-none-any.whl.metadata


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


### Message history with ChatMessageHistory

In [4]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import chain
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory


In [5]:
template = ChatPromptTemplate.from_messages([
     ('placeholder', "{history}"),
    ('human', "{prompt}")
])

chain = template | llm | StrOutputParser()

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


history = RunnableWithMessageHistory(
    chain, 
    get_session_history,
    input_messages_key="prompt",
    history_messages_key="history")

session_id = "Karthik"

get_session_history(session_id).clear()

response1 = history.invoke({"prompt": "What is the distance between earth and sun?"},
                          config = {"configurable": {"session_id": session_id}})

response2 = history.invoke({"prompt": "How about from moon"},
                          config = {"configurable": {"session_id": session_id}})

print(response1)
print("\n\n")
print(response2)



<think>
Okay, the user asked for "the distance between Earth and Sun." Let me start by recalling that this is a common question often related to astronomy basics. They probably want a straightforward answer but might not realize the complexity behind it.

First, I need to explain why there's no single fixed distance. The elliptical orbit means the distance changes throughout the year. I should mention perihelion and aphelion values so they know Earth isn't moving in a perfect circle. Then, introduce the Astronomical Unit (AU) as the standard unit here, since it's used across the solar system.

Wait, maybe the user is confused because numbers vary depending on when you ask. They might have heard different figures and want clarity. Also, they could be interested in how this distance affects seasons or climate, but didn't explicitly say that. Including a note about Earth being closer to perihelion during winter would help address deeper interests.

I should check sources for accurate numb

### ChatMessage History with SqlChatMessageHistory

In [6]:
from langchain_community.chat_message_histories import SQLChatMessageHistory


def get_session_history(session_id):
    return SQLChatMessageHistory(session_id=session_id, connection_string="sqlite:///chathistory.db")

template = ChatPromptTemplate.from_messages([
     ('placeholder', "{history}"),
    ('human', "{prompt}")
])

chain = template | llm | StrOutputParser()

history = RunnableWithMessageHistory(
    chain, 
    get_session_history,
    input_messages_key="prompt",
    history_messages_key="history")


session_id = "Karthik"

get_session_history(session_id).clear()

response1 = history.invoke({"prompt": "What is the distance between earth and sun?"},
                          config = {"configurable": {"session_id": session_id}})

response2 = history.invoke({"prompt": "How about from moon"},
                          config = {"configurable": {"session_id": session_id}})

print(response1)
print("\n\n")
print(response2)



C:\Users\kandd\AppData\Local\Temp\ipykernel_3532\3415544238.py:23: LangChainDeprecationWarning: `connection_string` was deprecated in LangChain 0.2.2 and will be removed in 1.0. Use connection instead.
  get_session_history(session_id).clear()


<think>
Okay, the user is asking for the distance between Earth and the Sun. Let me start by recalling that it's not a fixed number since both are moving. The average distance is about 149.6 million kilometers.

Wait, they might be looking for more than just the average because sometimes people don't know it varies. Should I mention the minimum and maximum distances? That adds context but maybe complicates things if they want simplicity. Let me check their possible scenario: Are they a student needing this for homework, or someone curious about astronomy basics?

They used "distance," so perhaps they expect the average figure first. But including the range could help them understand why it's not constant. Also, explaining that we use astronomical units (AU) might be useful since 1 AU is Earth's distance from the Sun by definition.

I should note that the exact number isn't a strict measurement but an average based on orbital mechanics. That shows they're learning about concepts beyond 